# Inspect export parameters from .edf files of an EEG database 
This notebook allows you to inspect the parameters of your .edf EEG database. \
It does not load and inspect the quality of your data. \
It helps you identify if you need to re-export some participant's data (due to poor signal resolution or clipping, as explained later). \
It provides information that helps analyze your dataset (such as the different channel labels). 

---
EDF (European Data Format) is a standard format for storing multichannel biological and physical signals: https://www.edfplus.info/. \
It was first published in 1992, and an upgraded version was released in 2003 (adding discontinuous recordings handling, annotations, stimuli, and events in a UTF-8 format). \
It is a compressed 16-bit format, meaning that each measured data point can take 2^16 values between the minimum and maximum values that you set at exportation. \
The min/max values correspond to the dynamic range of your data.

---
At the exportation in the .edf format (with software like Profusion from Compumedics), many parameters need to be set per recorded channels, such as channels' label, sampling frequency, filtering, units, and dynamic range. \
Exporting as .edf format is tedious and time-consuming, so mistakes in parameters can easily be made.  \
To avoid making mistakes, there is the possibility of implementing routines within some software (insert a link on how to make a routine in Compumedics). \
Inspecting those parameters can also be necessary if you work on an already existing dataset, to make sure that every participant's data is exploitable, specifically if the data comes from multiple sleep clinics. 

---
This notebook reads information from the .edf files, instead of loading the data and relying on existing packages. \
Existing packages, such as PYEDFLIB or MNE, are either too rigid (not able to read some data) or do not return all the information (such as boundaries of dynamic range, filtering parameters, etc).\
\
**To use this notebook, read each text box (called markdown cells in Jupyter notebook), run its associated code box (code cell), and read the output below.**\
To run a cell, select the cell and either click on "Run" and "Run Selected Cell" in the menu bar, or press "Shift+Enter". \
_The notebook will save summary tables as .tsv files (file format easy to read with Excel and to import in Python script) so that you can visually inspect (or reload later) if needed._ \
Those summaries will be stored in a summary folder within the study folder. \
\
This notebook is organized into 4 sections (with a section 0 to prepare your notebook):
1. Select your study folder, extract the data's information, and return general information
2. Inspect EEG channels
3. Inspect EOG channels
4. Inspect ECG channels

Once you are done with one section, you can reduce it by pressing the down arrow on the left of the section title, to improve the readability.

---
This notebook was developed on the ICEBERG database and tested on APOMORPHEE (from Noémie's internship).  \
last update 28/07/2025, YN

<hr style="height:4px; background-color:black; border:none;">

## 0. Import packages and define custom functions

In this notebook (and the majority of Python scripts), we use packages that contain pre-defined functions to perform our operations. \
The code cell below loads the packages required for this notebook and defines custom functions that we will use. \
If you have already installed all the packages, you will get the message ``Packages and functions successfully imported``. \
\
If you are missing one package, you will get an error ``ModuleNotFoundError: No module named 'my_package'``.\
To install the missing package, there are two solutions:
1. Install within Jupyter notebook:
    - open a new cell (click on "+" in the top-left toolbar of the notebook)
    - write ``%conda install nom_du_package --yes`` in the new cell
    - run the new cell ("Shift+Enter" or click "Run" and "Run Selected Cell" in the menu bar)
    - restart the kernel of the Jupyter notebook: click on "Kernel" and "Restart Kernel..."
    - re-run the import cell
<div style="margin-bottom:10px;"></div>

2. Install within a terminal: 
    - open a new terminal
    - enter the virtual environment you are working in (from where you installed Jupyter notebook) with ``conda activate my_virtual_environment``
    - run: ``conda install -k my_package``
    - restart the kernel of the Jupyter notebook: click on "Kernel" and "Restart Kernel..."
    - re-run the import cell

In [1]:
# Import cell
try:
    import os
    import re
    import chardet
    import warnings
    import traceback
    import numpy as np
    import pandas as pd
    from pathlib import Path
    import ipywidgets as widgets
    from ipyfilechooser import FileChooser
    from IPython.display import display, HTML, clear_output
except ImportError as e:
    print("⚠️ Error: ", e)
else:
    print("✅ Packages and functions successfully imported!")

# custom function to detect automatically and return the encoding of edf file
def detect_encoding(byte_string, min_confidence=0.6):
    result = chardet.detect(byte_string)
    encoding = result['encoding']
    confidence = result['confidence']
    if encoding is None or confidence < min_confidence:
        raise UnicodeDecodeError("chardet", byte_string, 0, len(byte_string),
                                 f"\tUnable to reliably detect encoding. Detected: {encoding} with confidence {confidence}")
    return encoding

# custom function to read information from EDF headers, without using the pyedflib package (that was too strict for ICEBERG)
# EDF file should follow a strict format, dedicating a specific number of octets for each type of information.
# it means that we can read the info octet by octet by specifying the number of octets we expect for the next variable (that is known from the EDF norm)
def read_edf_header_custom(file_path):
    with open(file_path, 'rb') as f: # open the file in binary mode, to read octet by octet. 
        header = {}
        # detect encoding
        raw_header = f.read(256)
        encoding = detect_encoding(raw_header)
        # print(f"\tDetected encoding for {file_path} : {encoding}")
        # Rewind to the beginning of the file
        f.seek(0)
        
        # the first 256 octets are global subject info
        header['version'] = f.read(8).decode(encoding).strip()
        header['patient_id'] = f.read(80).decode(encoding).strip()
        header['recording_id'] = f.read(80).decode(encoding).strip()
        header['start_date'] = f.read(8).decode(encoding).strip()
        header['start_time'] = f.read(8).decode(encoding).strip()
        header['header_bytes'] = int(f.read(8).decode(encoding).strip())
        header['reserved'] = f.read(44).decode(encoding).strip()
        header['n_data_records'] = int(f.read(8).decode(encoding).strip())
        header['duration_data_record'] = float(f.read(8).decode(encoding).strip())
        header['n_channels'] = int(f.read(4).decode(encoding).strip())
        
        # get info per channel
        n = header['n_channels']
        channel_fields = {
            'channel': [],
            'transducer_type': [],
            'dimension': [],
            'physical_min': [],
            'physical_max': [],
            'digital_min': [],
            'digital_max': [],
            'prefiltering': [],
            'sampling_frequency': [],
            'reserved': [],
        }

        for key in channel_fields:
            length = {
                'channel': 16,
                'transducer_type': 80,
                'dimension': 8,
                'physical_min': 8,
                'physical_max': 8,
                'digital_min': 8,
                'digital_max': 8,
                'prefiltering': 80,
                'sampling_frequency': 8,
                'reserved': 32,
            }[key]
            channel_fields[key] = [f.read(length).decode(encoding).strip() for _ in range(n)]

        header.update(channel_fields)
    
    return header

# function to extract filter information from the string in headers
def extract_filter_value(s, tag):
    if pd.isna(s):
        return None
    match = re.search(rf'{tag}[:\s]*([\d\.]+)\s*', s, re.IGNORECASE)
    return float(match.group(1)) if match else None

# custom function to get the sampling frequency out of a dataframe (the df needs to have 'subject' and 'channel' as columns)
def get_sf(df, subject, channel):
    df_sf = df[(df['subject'] == subject) & (df['channel'] == channel)]
    if not df_sf.empty:
        return df_sf.iloc[0]['sampling_frequency']
    else:
        return None

# function to create a widget slider to select the configuration to inspect
def mk_config_slider(value = 1, min = 1, max = 5):
    config_slider = widgets.IntSlider(
    value=value,
    min=min,
    max=max,
    step=1,
    description='Selected configuration:',
    style={'description_width': '150px'},   # increase description width (to adjust based on the description)
    layout=widgets.Layout(width='400px'),   # to adjust widget size
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
    )
    return config_slider

# function to print the configuration of a dataset parameter
def print_config(i, config_dict, param):
    # get the key and value from the dictionary
    idx = i - 1
    # get participant ID
    value = list(config_dict.values())  
    v = value[idx]  
    # get configuration
    key = list(config_dict.keys())
    k = key[idx]
    
    # print info
    print(f'Selected configuration: # {i}')
    print(f'\t{param}: {k}')
    print(f'\t{len(v)} participants: {v}')

# function to create a scrollable box for long output (e.g., cell loading the data) 
def print_in_scrollable_box(text, height=300, font_size="12px"):
    display(HTML(f'<pre style="overflow-y:scroll; height:{height}px; border:1px solid black; padding:10px; font-size:{font_size};">{text}</pre>'))


✅ Packages and functions successfully imported!


<hr style="height:4px; background-color:black; border:none;">

## 1. Select study folder, extract information, display general information 

### 1.1 Select the study folder

The code cell below will open a widget to select the folder containing your data.

In [2]:
# call widget to select your data folder 
chooser = FileChooser(os.getcwd())
chooser.title = "<b>Choose your study folder</b>"
chooser.show_only_dirs = True

# Define output widget to redirect the print within the function
out = widgets.Output()

# custom function to extract the folder path once the folder has been chosen: 
def on_folder_selected(chooser):
    with out:
        clear_output()
        chooser.folder_path = chooser.selected_path
        print("📁 Selected Path:", chooser.folder_path)
        
        # get the edf file list 
        chooser.edf_files = [
            f for f in Path(chooser.folder_path).rglob('*.edf')
            if not f.name.startswith('._') # don't select files starting with ._ (that can be found in mac for example)
            ]
        if not chooser.edf_files:
            print(f"⚠️ There is no .edf file in your folder")
        else:
            print(f"\nThere is {len(chooser.edf_files)} .edf files in your folder!")
        
        # check the existence and/or create the summary folder that will receive the summary tables and the report
        chooser.summary_path = f'{chooser.folder_path}/summary'
        if not os.path.exists(chooser.summary_path):
            os.makedirs(chooser.summary_path)
            print("\nCreated summary folder at: " + chooser.summary_path)
        else:
            print("\nSummary folder already exists. \nPrevious summary tables (if any) will be overwritten at: \n" + chooser.summary_path)

# callback to run the function only when a folder is selected
chooser.register_callback(on_folder_selected)
display(chooser, out)

FileChooser(path='C:\Users\yvan.nedelec\OneDrive - ICM\Documents\RE_yvan\projects\Dream-Toolkit\python\Check_E…

Output()

### 1.2 Extract infromation from each file parameters from each participant

The code cell below will loop across the .edf files to extract the information of each subject.\
It returns a table that can easily be manipulated to access specific information in the rest of the notebook.\
If some files failed to load, their names will be saved in a .tsv file in the summary folder.

In [3]:
# get variables from the chooser widget
folder_path = chooser.folder_path
summary_path = chooser.summary_path
edf_files = chooser.edf_files

# check if there is a participants.tsv file to get different groups or sessions
# if there is not a participants.tsv we will try to infer groups from subfolder organization or filename components (additional part from subject number)
# in ICEBERG, subfolders define groups within the data folder
# in APOMORPHEE, suffixes define nights ("session") 
table_found = False
for root, dirs, files in os.walk(folder_path):
    if 'participants.tsv' in files:
        table_found = True        
        print(f"Table containing participants information found at: {os.path.join(root, 'participants.tsv')} ")
        subj_table_path = os.path.join(root, 'participants.tsv')

found_group = False
if table_found:
    subj_table = pd.read_csv(subj_table_path, sep = '\t', dtype={'participant_id': str, 'group': str})
    if "group" in subj_table.columns:
        found_group = True
        print(f"We will extract participant's group from it")
    else:
        print(f"No column 'group' was found in the table")
        print(f"Group will be inferred from subfolder organization or subfolder component") 
    
else:
    print(f"No table containing participants information (labelled 'participants.tsv') was found")
    print(f"If you have a table, please rename it 'participants.tsv' (and make sure you have columns labelled 'participant_id' and 'group' if any)")
    print(f"In the meantime, we will infer participant's group from subfolder organization or filename component in the next cells")
    subj_table = pd.DataFrame()

# Initialize a list of dataframes to store file info, which will be concatenated at the end (this is better for performance)
df_list = []
# Initialize an empty list for files that could not be read
failed_list = []
output = ""

# intialize a dynamic output
dynamic_out = widgets.Output()
display(dynamic_out)

for e, edf_path in enumerate(edf_files):
    with dynamic_out:
        output += (f'file {e+1}/{len(edf_files)}, currently opening file: {edf_path}\n')
        dynamic_out.clear_output(wait=True)
        print_in_scrollable_box(output, font_size = "12px")
        
        # read file with the custom function
        try:
            edf_header = read_edf_header_custom(edf_path) 
            
            # get subject name (corresponding to file_name)
            sub_name = edf_path.stem
            
            # get subject group (from the parent folder because in the ICEBERG database subfolders were created per patient group)
            sub_folder = edf_path.parent.name # get the parent folder of the subject file (path)
            
            # create df from signal info
            df = pd.DataFrame(edf_header)
                
            # theoretical resolution (edf are 16bit files so the eeg signal can take 2^16 values within the dynamic range)
            df['res_theoretical'] = (abs(pd.to_numeric(df['physical_min']))+abs(pd.to_numeric(df['physical_max'])))/pow(2,16)
            # turn theoretical resolution to uV if dimension is mV (if no dimension, it is a mess)
            df.loc[df['dimension'].str.contains('mv', case=False, na=False), 'res_theoretical'] *= 1000
            
            # get filtering info in different columns
            df['lowpass']   = df['prefiltering'].apply(lambda x: extract_filter_value(x, 'LP'))
            df['highpass']  = df['prefiltering'].apply(lambda x: extract_filter_value(x, 'HP'))
            df['notch']  = df['prefiltering'].apply(lambda x: extract_filter_value(x, 'NOTCH'))
            
            # add subject info in the dataframe
            df['subject'] = sub_name
            df['sub_folder'] = sub_folder
            df['group'] = np.nan # initialyze column 'group' with NaN
            # get group from participants table if any (else group will be inferred from subfolder or filename extension later)
            if found_group:
                df['group'] = subj_table.loc[subj_table['participant_id'] == sub_name, 'group'].iloc[0]

            # extract filename component before and after subject number (so we assume subject name contains at least incrementing numbers that are at the beginning of the file name)  
            #   ^       → start of string  
            # (.*?)     → group 1: as few chars as possible, up to the first digit  
            # (\d+)     → group 2: the number itself  
            # (.*)      → group 3: the rest of the string  
            # $         → end of string
            pattern = re.compile(r'^(.*?)(\d+)(.*)$')
            m = pattern.match(sub_name)
            if m:
                pre_comp = m.group(1) or np.nan
                sub_num = m.group(2) or np.nan
                post_comp = m.group(3) or np.nan
            df['pre_fn_comp'] = pre_comp
            df['post_fn_comp'] = post_comp
            df['sub_num'] = sub_num
            
            df['path'] = str(edf_path)
            df['session'] = np.nan # session will be inferred later from file name component
            
            # select only the columns of interest
            df = df[['subject', 'group', 'session', 'path', 'sub_folder', 'sub_num', 'pre_fn_comp', 'post_fn_comp', 'channel', 'transducer_type', 'dimension', 'sampling_frequency', 
                 'highpass', 'lowpass', 'notch', 'physical_min', 'physical_max', 'res_theoretical']]
            
            # store subject data
            df_list.append(df)
    
        except UnicodeDecodeError as e:
            err = f"⚠️ Encoding problem for {edf_path}\n"
            output += err
            clear_output(wait=True)
            print_in_scrollable_box(output, font_size="12px")
            failed_list.append((edf_path, 'encoding'))
        except Exception as e:
            # tb = traceback.format_exc()
            err = f"❌ Unexpected problem for {edf_path} : {e}\n"
            output += err
            clear_output(wait=True)
            print_in_scrollable_box(output, font_size="12px")
            failed_list.append((edf_path, 'other'))
   
# concatenate dataframe into one and only
with warnings.catch_warnings(): # this is to skip a warning not affecting our operation
    warnings.simplefilter("ignore", FutureWarning)
    df_full = pd.concat(df_list, ignore_index=True)

# save the failed list if not empty:
failed_df = pd.DataFrame(failed_list)
if not failed_df.empty:
    failed_df.to_csv(f'{summary_path}/failed_edf_read.tsv', sep = '\t')
    print(f'\nSaving the list of files that could not be read to: \n{summary_path}/failed_edf_read.tsv')    

No table containing participants information (labelled 'participants.tsv') was found
If you have a table, please rename it 'participants.tsv' (and make sure you have columns labelled 'participant_id' and 'group' if any)
In the meantime, we will infer participant's group from subfolder organization or filename component


Output()

### 1.3 Dataset general information (# participants, groups, recorded sensors)

In the code cell below, we extract group and session information, and we save the summary table (in the summary folder) so that it can be visually inspected with Excel, or reloaded in other script if needed.\
\
To extract group infromation:
- we check if group was read from participants.tsv
- else we try to infer it from subfolders (e.g. ICEBERG databaase organization)
- else we try to infer it from filename component (characters before or after the participant number in the filename)  
- else we consider there is only one group in the study

To extract session information: 
- we infer it from filename components (if there is more than one filename component per participant)
- else we consider there is only one session per participant

Then, we display the general information of the dataset (# files, # participants, # groups, # sessions)

In [6]:
# get group information, from participants.tsv file, sub_folder, or filename component
print("Get group information:")
if df_full['group'].isna().all():
    print("The column 'group' is empty, we will infer group from subfolder, if any...")
    if len(df_full['sub_folder'].unique()) > 1:
        df_full['group'] = df_full['sub_folder']
        print(">>> Group inferred from folders within the database <<<")
    else:
        print("There is no distinct folders for groups.")
        print("Trying to infer group from filename component...")
        # looping across subject number (and not subject filename) to test if there are multiple filename components per subject (to disentangle groups from session)  
        count_precomp = np.zeros(len(df_full['sub_num'].unique()))
        count_postcomp = np.zeros(len(df_full['sub_num'].unique()))
        for sn, sub_num in enumerate(df_full['sub_num'].unique()):
            df_sub = df_full[df_full['sub_num'] == sub_num]
            count_precomp[sn] = len(df_sub['pre_fn_comp'].unique())
            count_postcomp[sn] = len(df_sub['post_fn_comp'].unique())
        # fn component is a group if within subject there is only one component, but there are multiple components between subject
        # 1st, try for component before the subject number, 2nd try for component after the subject number 
        if len(df_full['pre_fn_comp'].unique()) > 1 and count_precomp.mean() == 1:
            df_full['group'] = df_full['pre_fn_comp']
            print(">>> Group inferred from filename component (before subject number) <<<")
        elif len(df_full['post_fn_comp'].unique()) > 1 and count_postcomp.mean() == 1:
            df_full['group'] = df_full['post_fn_comp']
            print(">>> Group inferred from filename component (after subject number) <<<")
        else:
            print("Did not succed to identify group from filename component.")
            print("It seems that there is only one group in the study!")
else:
    print(">>> Group information coming from participants.tsv <<<")

print("\nGet session information")
if len(df_full['pre_fn_comp'].unique()) > 1 and count_precomp.mean() > 1:
    print(">>> Session inferred from filename component (before subject number) <<<")
    df_full['session'] = df_full['pre_fn_comp']
elif len(df_full['post_fn_comp'].unique()) > 1 and count_postcomp.mean() > 1:
    print(">>> Session inferred from filename component (after subject number) <<<")
    df_full['session'] = df_full['post_fn_comp']
else:
    print("It seems that there is only one session in the study")

# save summary table containing full info
df_full.to_csv(f'{summary_path}/FULL_summary_table_edf.tsv', sep = '\t')
print(f'\nSaving full informations from the dataset to:\n{summary_path}/FULL_summary_table_edf.tsv')

print("\n\nDataset information:")
print(f"- Number of files: {len(df_full['subject'].unique())}")
print(f"- Number of participants: {len(df_full['sub_num'].unique())}")
print(f"- Number of groups: {len(df_full['group'].unique())}")
print(f"- Number of sessions: {len(df_full['session'].unique())}")

if len(df_full['group'].unique()) > 1:
    print("\nParticipants per groups:")
    print(df_full.drop_duplicates().groupby('group').agg(n_subjects=('subject', 'nunique')))

Get group information:
The column 'group' is empty, we will infer group from subfolder, if any...
There is no distinct folders for groups.
Trying to infer group from filename component...
Did not succed to identify group from filename component.
It seems that there is only one group in the study!

Get session information
>>> Session inferred from filename component (after subject number) <<<

Saving full informations from the dataset to:
C:\Users\yvan.nedelec\OneDrive - ICM\Documents\RE_yvan\projects\noemie_rescue\data/summary/FULL_summary_table_edf.tsv


Dataset information:
- Number of files: 66
- Number of participants: 33
- Number of groups: 1
- Number of sessions: 2


In the code cell below, we print the unique values of the sensors whithin your database, in case you want to make sure that one specific type of sensors is in your dataset. \
For the rest of the notebook, we will focus, separately, on EEG, EOG and ECG sensors

In [7]:
print('\nFull recorded sensors configuration of your database (across participants): ')
ch_output = '\n'.join(df_full['channel'].unique())
print_in_scrollable_box(ch_output, height = 150)


Full recorded sensors configuration of your database (across participants): 


<hr style="height:4px; background-color:black; border:none;">

## 2. Inspect EEG

### 4.1 Select only the EEGs

In [10]:
# select only EEG channels and return a warning if the number of participant is smaller/higher
mask_ch = df_full['transducer_type'].str.contains(r'EEG|AGAGCL ELECTRODE', case = False, na=False) # create a mask that returns true for lines containing either EEG/AGAGCL ELECTRODE in the transducer_type column
df_ch = df_full[mask_ch]
# remove the emg channels that were captured with the AGAGCL ELECTRODE transducer type 
df_ch = df_ch[~df_ch['channel'].str.contains(r'emg|ecg|eog', case=False, na=False)] # the ~ allows to not select the selection (like ! in matlab)

# Check if the number of participants with only EEG is the same as df_full. 
# If not, it might be because the transducer type was no correctly detected. 
# One possibility is to add the type of transducer to the condition line 2 of this cell.
if len(df_full['subject'].unique()) > len(df_ch['subject'].unique()):
    # identify missing subjects
    missing_sub = set(df_full['subject'].unique()) - set(df_ch['subject'].unique())
    print('\n!!! There is less participants in the dataset with only EEGs !!!')
    print(f'Missing participants: {missing_sub}')
    print("\nEither these participants don't have EEGs.")
    print("Or the transducer type was not correctly detected.")
    # get df of missing sub to save and inspect
    df_miss = df_full[df_full['subject'].isin(missing_sub)]
    df_miss.to_csv(f'{summary_path}/EEG_missing_edf.tsv', sep = '\t')
    print(f'\nSaving informations from missing participants to:\n{summary_path}/EEG_missing_edf.tsv')
    print('Please inspect the file, and specifically the column transducer_type')
elif len(df_full['subject'].unique()) < len(df_ch['subject'].unique()):
    print('\n!!! There is more participants in the dataset with only EEGs !!!')
    print('This should not be the case.')
    print('Please inspect what is happening in a code editor (spyder..), or ask Yvan.')
    more_sub = set(df_ch['subject'].unique()) - set(df_full['subject'].unique())
    df_more = df_ch[df_ch['subject'].isin(more_sub)]
    df_more.to_csv(f'{summary_path}/EEG_suspect_edf.tsv', sep = '\t')
    print(f'\nSaving informations from suspect participants to:\n{summary_path}/EEG_suspect_edf.tsv')

# saving info from eeg
df_ch.to_csv(f'{summary_path}/EEG_summary_table.tsv', sep = '\t')
print(f'\nSaving informations from EEGs to:\n{summary_path}/EEG_summary_table.tsv')


Saving informations from EEG channels to:
C:\Users\yvan.nedelec\OneDrive - ICM\Documents\RE_yvan\projects\ICEBERG\PSG/summary/EEG_summary_table.tsv


### 4.2 Inspect EEG configurations

There will be likely many EEG configurations (especially with multicentric dataset).  \
\
For polysomnographic EEG, there should be at least:
- 4 EEG (Fp1, C3, O1 and A2 that will be used as the reference) (or less frequently Fp2, C4, O2 and A1)

Depending on the analysis you plan, if one configuration does not contain those electrodes you will need either to re-export the data or to exclude the participant. \
\
If the EEG label is Fp1-A2, it means that your data is already re-referenced to A2.

In [11]:
# get the EEG configuration per participant 
ch_per_sub = df_ch.groupby('subject')['channel'].apply(lambda x: tuple(sorted(set(x))))

# identify the channel configuration of each participant and store them in a dict to print per channel config
ch_config_dict = {}
for config in ch_per_sub.unique():
    sub = ch_per_sub[ch_per_sub == config].index.tolist()
    ch_config_dict[config] = sub

if len(ch_config_dict) > 1:
    print('\n>>> There is multiple EEG configurations in your dataset! <<<')    
    print(f'\n\tNumber of different configuration: {len(ch_config_dict)}\n')
else:
    print('\n>>> There is only one EEG configuration in your dataset! <<<')

# # print info per channel configuaration
# for i, (config, participants) in enumerate(ch_config_dict.items(), 1):
#     print(f'Configuration #{i} ({len(participants)} participants):')
#     print(f'Channels ({len(config)}) : {config}\n')


>>> There is multiple EEG configurations in your dataset! <<<

	Number of different channels configuration: 11



To inspect the participants ID in one configuration, run the cell just below after changing the parameter  "configuration_to_inspect".

In [12]:
# widget to select the configuration of interest
config_ch_slider = mk_config_slider(value = 1, min = 1, max = len(ch_config_dict))

# print the configuration selected
# interact with the slider output through the printing function 
widgets.interact(lambda i: print_config(i, config_dict=ch_config_dict, param="Channels"), i=config_ch_slider);

interactive(children=(IntSlider(value=1, continuous_update=False, description='Selected configuration:', layou…

### 4.3 Inspect EEG sampling frequency

Ideally, you expect to have only one sampling frequency for all the EEGs and participants. \
In practice, you might have different sampling frequencies across participants (especially with multicentric dataset).\
\
Each EEG analysis software handles multiple sampling frequencies (that can happen for EEG and EOG) within participants differently. For example:
- MNE python will automatically upsample channels to the highest sampling frequency
- Fieldtrip will load only a subset of channels (with the sampling frequency the most represented)   

In [13]:
# the sampling frequency configuration
sf_per_sub = df_ch.groupby('subject')['sampling_frequency'].apply(lambda x: tuple(sorted(set(x))))
# identify the sampling frequency configuration of each participant and store them in a dict to print per sampling configuration config
sf_config_dict = {}
for config in sf_per_sub.unique():
    sub = sf_per_sub[sf_per_sub == config].index.tolist()
    sf_config_dict[config] = sub

# print info per sf configuration (maybe print it only for multiple config)
if len(sf_config_dict) > 1:
    print('\n>>> There is multiple sampling frequency for EEGs in your dataset! <<<')    
    print(f'\n\tNumber of different sampling frequency configuration: {len(sf_config_dict)}\n')
    print('Quick overlook of the EEGs associated to sampling frequencies:')
    for s, sf in enumerate(df_ch['sampling_frequency'].unique()):
        # select only rows with the current sf
        df_sf = df_ch[df_ch['sampling_frequency'] == sf].copy()
        print(f'\n{sf} Hz: {df_sf["channel"].unique()}')
else:
    print(f'\n>>> There is only one sampling frequency for EEGs in your dataset: {df_ch['sampling_frequency'].unique()} <<<')

# print('\nSampling frequency configurations:\n')
# for i, (config, participants) in enumerate(sf_config_dict.items(), 1):
#     print(f'Configuration #{i} ({len(participants)} participants):')
#     print(f'Sampling frequency ({len(config)}) : {config}\n')


>>> There is multiple sampling frequency for EEGs in your dataset! <<<

	Number of different sampling frequency configuration: 2

Quick overlook of the EEGs associated to sampling frequencies:

256 Hz: ['Fp1' 'Fp2' 'F3' 'T7/T3' 'C3' 'Cz' 'C4' 'T8/T4' 'A1' 'A2' 'O1' 'O2' 'F7'
 'F4' 'P7/T5' 'P3' 'P4' 'P8/T6' 'Pz' 'Fp1-A2' 'C3-A2' 'O1-A2' 'T3' 'T4']

400 Hz: ['FP1-A2' 'C3-A2' 'O1-A2' 'FP2-A1' 'C4-A1' 'O2-A1' 'T3-A2' 'T4-A1']


To inspect the participants ID in one configuration, run the cell just below after changing the parameter  "configuration_to_inspect".

In [14]:
# widget to select the configuration of interest
config_sf_slider = mk_config_slider(value = 1, min = 1, max = len(sf_config_dict))

# print the configuration selected
# interact with the slider output through the printing function 
widgets.interact(lambda i: print_config(i, config_dict=sf_config_dict, param="Sampling frequencies"), i=config_sf_slider);

interactive(children=(IntSlider(value=1, continuous_update=False, description='Selected configuration:', layou…

### 4.4 Inspect EEG filtering parameters

Ideally with EEG sleep data, you want no lowpass nor notch filter, and a very low highpass filter around 0.05 Hz (to remove slow drift in long recordings).

In [15]:
if len(df_ch['highpass'].unique())+len(df_ch['lowpass'].unique())+len(df_ch['notch'].unique()) == 3:
    print('\n>>> All EEGs have the same filtering parameters! <<<')
elif len(df_ch['highpass'].unique())+len(df_ch['lowpass'].unique())+len(df_ch['notch'].unique()) > 3:
    print('\n>>> Filtering parameters are not fully consistent in EEGs across the dataset! <<<')
else:
    print('\n>>> There may have been a problem in reading the filtering parameters. Here is the output: <<<')

# Get the list of participants with different filtering parameters
# 1st replace NaN because groupby does not like NaN
df_filt = df_ch.copy()
df_filt[['lowpass', 'highpass', 'notch']] = df_filt[['lowpass', 'highpass', 'notch']].fillna('missing')

config_filters = (
    df_filt.groupby(['lowpass', 'highpass', 'notch'])['subject']
    .apply(lambda x: sorted(set(x)))
    .reset_index(name = 'subjects')
)

# print filter configuration
print(f'\n\tNumber of different EEG filters configurations: {len(config_filters)}\n')
# print('\nFilters configurations: ')
# r=1
# for row in config_filters.itertuples(index=False):
#     print(f'Configuration #{r} ({len(row.subjects)} participants)')
#     print(f'highpass: {row.highpass}, lowpass: {row.lowpass}, notch: {row.notch}\n')
#     r=r+1



>>> Filtering parameters are not fully consistent in EEGs across the dataset! <<<

	Number of different EEG filters configurations: 3



To inspect the participants ID in one configuration, run the cell just below after changing the parameter  "configuration_to_inspect".

In [16]:
# widget to select the configuration of interest
config_filter_slider = mk_config_slider(value = 1, min = 1, max = len(config_filters))

# function to rpint filters configurations
def print_filters(config_slider):
    # get the info from the dataframe
    idx = config_slider - 1
    sID = config_filters.iloc[idx]['subjects']
    hpass = config_filters.iloc[idx]['highpass']
    lpass = config_filters.iloc[idx]['lowpass']
    notch = config_filters.iloc[idx]['notch']
    
    # print info
    print(f'Selected configuration: # {config_slider}')
    print(f'\tFilters configuration: highpass: {hpass}; lowpass: {lpass}; notch: {notch}')
    print(f'\t{len(sID)} participants: {sID}')

widgets.interact(print_filters, config_slider = config_filter_slider);

interactive(children=(IntSlider(value=1, continuous_update=False, description='Selected configuration:', layou…

### 4.5 Inspect EEG units

At the exportation, channels can be imported in different units. \
Each analysis software will handle units differently, so it can be helpful to know which units your dataset contains. 
- MNE python will automatically detect the units and convert the data to Volt. However, if the unit is not read correctly, the data will **not** be converted (e.g. "UV" is not interpredted as uV, therefore data are not converted to Volt )
- fieldtrip is loading the data with their unit of origin, so you might want to convert all channels to the same unit before your analysis  

In [17]:
if len(df_ch['dimension'].unique()) == 1:
    print(f'\n>>> All EEGs have the same unit: {df_ch["dimension"].unique()} <<<\n')
elif len(df_ch['dimension'].unique()) > 1:
    print('\n>>> Multiple units were found! <<<')
    print(f'\n\tNumber of different units configurations: {len(df_ch['dimension'].unique())}\n')
    print('Quick overlook of EEGs associated to units:')
    for u, unit in enumerate(df_ch['dimension'].unique()):
        # select only rows with the current sf
        df_unit = df_ch[df_ch['dimension'] == unit].copy()
        print(f'\n{unit}: {df_unit["channel"].unique()}')
    

# print the different configuration of units 
# if info about sf configuration is needed
unit_per_sub = df_ch.groupby('subject')['dimension'].apply(lambda x: tuple(sorted(set(x))))
ch_per_unit = df_ch.groupby('dimension')['channel'].apply(lambda x: tuple(sorted(set(x))))
# identify the sampling frequency configuration of each participant and store them in a dict to print per sampling configuration config
unit_config_dict = {}
for config in unit_per_sub.unique():
    sub = unit_per_sub[unit_per_sub == config].index.tolist()
    unit_config_dict[config] = sub

# print info per sf configuration
# print('\nUnits configurations:')
# for i, (config, participants) in enumerate(unit_config_dict.items(), 1):
#     print(f'Configuration #{i} ({len(participants)} participants):')
#     print(f'Unit ({len(config)}) : {config}\n')
#     # print(f"Participants : {participants}\n")


>>> Multiple units were found! <<<

	Number of different units configurations: 2

Quick overlook of channels associated to units:

uV: ['Fp1' 'Fp2' 'F3' 'T7/T3' 'C3' 'Cz' 'C4' 'T8/T4' 'A1' 'A2' 'O1' 'O2' 'F7'
 'F4' 'P7/T5' 'P3' 'P4' 'P8/T6' 'Pz' 'Fp1-A2' 'C3-A2' 'O1-A2' 'T3' 'T4']

UV: ['FP1-A2' 'C3-A2' 'O1-A2' 'FP2-A1' 'C4-A1' 'O2-A1' 'T3-A2' 'T4-A1']


To inspect the participants ID in one configuration, run the cell just below after changing the parameter  "configuration_to_inspect".

In [18]:
# widget to select the configuration of interest
config_unit_slider = mk_config_slider(value = 1, min = 1, max = len(unit_config_dict))

# print the configuration selected
# interact with the slider output through the printing function 
widgets.interact(lambda i: print_config(i, config_dict=unit_config_dict, param="Units"), i=config_unit_slider);

interactive(children=(IntSlider(value=1, continuous_update=False, description='Selected configuration:', layou…

### 4.6 Inspect EEG signal inversion

Some softwares (e.g. profusion from compumedics) allows to invert the polarity of the exported data. It can be extremely confusing and can lead to wrong results. \
Here, we inspect if the signal is inverted by checking if the minimum physical boundary is higher than the maximum physical boundary. \
For .edf file, the physical boundaries are values that are set when exporting the data by specifying the scale of the data. \
\
In profusion (from compumedics) a scale of 1mV will lead to a min physical boundary of -500 uV and a max physical boundary of +500 uV.\
\
For other EEG format and software, the dynamical range might be set before recording (e.g. to be specified in the montage) and can't be changed at the exportation.

In [19]:
# select rows where the physical min is greater than the physical max
df_inv = df_ch[df_ch['physical_min'] > df_ch['physical_max']]

if not df_inv.empty:
    print('\n>>> Inverted polarity detected in EEGs! <<<')
    print(f'{df_inv.shape[0]} EEGs have an inverted polarity (from {df_ch.shape[0]} EEGs in {len(edf_files)} edf files)')
    print(df_inv[['subject', 'channel', 'dimension', 'physical_min', 'physical_max']])
else:
    print('\n>>> No inverted polarity was detected in EEGs <<<')
df_inv.to_csv(f'{summary_path}/EEG_inverted_polarity_edf.tsv', sep = '\t')
print(f'\nSaving informations from inverted polarity EEGs to:\n{summary_path}/EEG_inverted_polarity_edf.tsv \n(will be empty if no inverted polarity)')



>>> No inverted polarity was detected in EEGs <<<

Saving informations from EEG inverted polarity channels to:
C:\Users\yvan.nedelec\OneDrive - ICM\Documents\RE_yvan\projects\ICEBERG\PSG/summary/EEG_inverted_polarity_edf.tsv 
(will be empty if no inverted polarity)


### 4.7 Inspect EEG theoretical resolution

The theoretical resolution of .edf file is the minimum amplitude variation that can be recorded between two samples. \
Since .edf file is a 16-bit compressed format (meaning that a datapoint can take 2^16 value between a min and a max), we computed the theoretical resolution by dividing the dynamical range (upper/lower physical boundaries)  by 2^16. \
Ideally, resolution should be around 0.01 uV (or lower). \
\
Hence, to improve the theoretical resolution, we can reduce the min and max values of the dynamic range. However, reducing the dynamic range can lead to loss of data information (because the signal can't get higher or lower than the boundaries), an issue called signal clipping.\
\
Below, we detect EEGs that have a resolution higher than 0.1 uV.\
You can change the resolution threshold with the widget:

In [20]:
# res_theo have been converted to uV, but if dimension was not read or not indicated in the headers, it might not work. I might need to add something more robust
r_thres = widgets.BoundedFloatText(
    value=0.1,
    min=0,
    max=10.0,
    step=0.1,
    style={'description_width': '150px'},  # augmente la largeur de la description
    layout=widgets.Layout(width='230px'),   # ajuste la taille totale du widget si besoin
    description='Resolution threshold (uV):',
    disabled=False
);

# define a function to interact with the widget
def check_bad_res(threshold):
    r_mask = df_ch['res_theoretical'] >= threshold
    bad_res = df_ch[r_mask]
    
    if not bad_res.empty:
        print(f'\n>>> Poor resolution detected in EEGs! (>= {threshold} uV) <<<')
        print(f'{bad_res.shape[0]} EEGs have a very poor resolution (from {df_ch.shape[0]} EEGs in {len(edf_files)} edf files)')
        print(bad_res[['subject', 'channel', 'dimension', 'physical_min', 'physical_max', 'res_theoretical']])
    else:
        print(f'\n>>> No EEG with a poor resolution (>= {threshold} uV) was detected! <<<')
    bad_res.to_csv(f'{summary_path}/EEG_bad_resolution_edf.tsv', sep = '\t')
    print(f'\nSaving informations from bad resolution EEGs to:\n{summary_path}/EEG_bad_resolution_edf.tsv \n(will be empty if no bad resolution)')

widgets.interact(check_bad_res, threshold=r_thres);

interactive(children=(BoundedFloatText(value=0.1, description='Resolution threshold (uV):', layout=Layout(widt…

### 4.8 Inspect EEG dynamic range

As mentionned above, a small dynamic range will lead to a good signal resolution, but can lead to signal clipping.\
Signal clipping happens when the signal reach the physical boundaries (min or max) and therefore is blocked at this value. It results in a loss of data information.\
\
Typical physiological EEG data (good quality) varies from +/-200 uV.\
Below, we check if the dynamic range physical boundaries are lower than 400 uV (+/- 200 uV). \
You can change the dynamic range threshold with the widget:

In [21]:
dr_thres = widgets.BoundedFloatText(
    value=400,
    min=0,
    max=5000,
    step=0.1,
    style={'description_width': '200px'},  # augmente la largeur de la description
    layout=widgets.Layout(width='270px'),   # ajuste la taille totale du widget si besoin
    description='Dynamic range threshold (uV):',
    disabled=False
);


def check_bad_dr(threshold):
    dr_mask = df_ch['res_theoretical']*pow(2,16) <= threshold
    bad_dr = df_ch[dr_mask]
    
    if not bad_dr.empty:
        print(f'\n>>> Small dynamic range detected in EEGs! (<= {threshold} uV) <<<\n')
        print(f'{bad_dr.shape[0]} EEGs have a small dynamic range (from {df_ch.shape[0]} EEGs in {len(edf_files)} edf files)')
        print(bad_dr[['subject', 'channel', 'dimension', 'physical_min', 'physical_max', 'res_theoretical']])
    else:
        print(f'\n>>> No EEG with a small dynamic range(<= {threshold} uV) was detected! <<<')
    bad_dr.to_csv(f'{summary_path}/EEG_bad_dynamic_range_edf.tsv', sep = '\t')
    print(f'\nSaving informations from bad dynamic range EEGs to:\n{summary_path}/EEG_bad_dynamic_range_edf.tsv \n(will be empty if no bad resolution)')

widgets.interact(check_bad_dr, threshold = dr_thres);

interactive(children=(BoundedFloatText(value=400.0, description='Dynamic range threshold (uV):', layout=Layout…

## 5. Inspect EOG

### 5.1 Select only EOGs

In [22]:
# select only EOGs and return a warning if the number of participant is smaller/higher
mask_eog = df_full['channel'].str.contains(r'EOG', case = False, na=False) # create a mask that returns true for lines containing either EOG in the channel column
df_eog = df_full[mask_eog]
# remove the emg channels that were captured with the AGAGCL ELECTRODE transducer type 
# df_eog = df_eog[~df_eog['channel'].str.contains(r'emg|ecg|eeg|a1|a2', case=False, na=False)] # the ~ allows to not select the selection (like ! in matlab)

# Check if the number of participants with only EOG is the same as df_full. 
# If not, it might be because the transducer type was no correctly detected. 
# One possibility is to add the type of transducer to the condition line 2 of this cell.
if len(df_full['subject'].unique()) > len(df_eog['subject'].unique()):
    # identify missing subjects
    missing_sub = set(df_full['subject'].unique()) - set(df_eog['subject'].unique())
    print('\n!!! There is less participants in the dataset with only EOGs !!!')
    print(f'Missing participants: {missing_sub}')
    print("\nEither these participants don't have EOGs.")
    print("Or the transducer type was not correctly detected.")
    # get df of missing sub to save and inspect
    df_eogmiss = df_full[df_full['subject'].isin(missing_sub)]
    df_eogmiss.to_csv(f'{summary_path}/EOG_missing_edf.tsv', sep = '\t')
    print(f'\nSaving informations from missing participants to:\n{summary_path}/EOG_missing_edf.tsv')
    print('Please inspect the file, and specifically the column transducer_type')
elif len(df_full['subject'].unique()) < len(df_eog['subject'].unique()):
    print('\n!!! There is more participants in the dataset with only EOGs !!!')
    print('This should not be the case.')
    print('Please inspect what is happening in a code editor (spyder..), or ask Yvan.')
    more_sub = set(df_eog['subject'].unique()) - set(df_full['subject'].unique())
    df_more = df_eog[df_eog['subject'].isin(more_sub)]
    df_more.to_csv(f'{summary_path}/EOG_suspect_edf.csv', sep = '\t')
    print(f'\nSaving informations from suspect participants to:\n{summary_path}/EOG_suspect_edf.tsv')

# saving info from EOG
df_eog.to_csv(f'{summary_path}/EOG_summary_table.tsv', sep = '\t')
print(f'\nSaving informations from EOGs to:\n{summary_path}/EOG_summary_table.tsv')


Saving informations from EOG channels to:
C:\Users\yvan.nedelec\OneDrive - ICM\Documents\RE_yvan\projects\ICEBERG\PSG/summary/EOG_summary_table.tsv


### 5.2 Inspect EOG configurations

In [23]:
# get the EOG configuration per participant 
ch_per_sub = df_eog.groupby('subject')['channel'].apply(lambda x: tuple(sorted(set(x))))

# identify the EOG configuration of each participant and store them in a dict to print per EOG config
ch_config_dict = {}
for config in ch_per_sub.unique():
    sub = ch_per_sub[ch_per_sub == config].index.tolist()
    ch_config_dict[config] = sub

if len(ch_config_dict) > 1:
    print('\n>>> There is multiple EOG configurations in your dataset! <<<')    
    print(f'\n\tNumber of different configuration: {len(ch_config_dict)}\n')
else:
    print('\n>>> There is only one EOG configuration in your dataset! <<<')

# # print info per channel configuaration
# for i, (config, participants) in enumerate(ch_config_dict.items(), 1):
#     print(f'Configuration #{i} ({len(participants)} participants):')
#     print(f'Channels ({len(config)}) : {config}\n')


>>> There is multiple EOG configurations in your dataset! <<<

	Number of different channels configuration: 2



In [24]:
# widget to select the configuration of interest
config_ch_slider = mk_config_slider(value = 1, min = 1, max = len(ch_config_dict))

# print the configuration selected
# interact with the slider output through the printing function 
widgets.interact(lambda i: print_config(i, config_dict=ch_config_dict, param="Channels"), i=config_ch_slider);

interactive(children=(IntSlider(value=1, continuous_update=False, description='Selected configuration:', layou…

### 5.3 Inspect EOG sampling frequencies

In [25]:
# the sampling frequency configuration
sf_per_sub = df_eog.groupby('subject')['sampling_frequency'].apply(lambda x: tuple(sorted(set(x))))
# identify the sampling frequency configuration of each participant and store them in a dict to print per sampling configuration config
sf_config_dict = {}
for config in sf_per_sub.unique():
    sub = sf_per_sub[sf_per_sub == config].index.tolist()
    sf_config_dict[config] = sub

# print info per sf configuration (maybe print it only for multiple config)
if len(sf_config_dict) > 1:
    print('\n>>> There is multiple sampling frequency for EOGs in your dataset! <<<')    
    print(f'\n\tNumber of different sampling frequency configuration: {len(sf_config_dict)}\n')
    print('Quick overlook of the EOGs associated to sampling frequencies:')
    for s, sf in enumerate(df_eog['sampling_frequency'].unique()):
        # select only rows with the current sf
        df_sf = df_eog[df_eog['sampling_frequency'] == sf].copy()
        print(f'\n{sf} Hz: {df_sf["channel"].unique()}')
else:
    print(f'\n>>> There is only one sampling frequency for EOGs in your dataset: {df_eog['sampling_frequency'].unique()} <<<')

# print('\nSampling frequency configurations:\n')
# for i, (config, participants) in enumerate(sf_config_dict.items(), 1):
#     print(f'Configuration #{i} ({len(participants)} participants):')
#     print(f'Sampling frequency ({len(config)}) : {config}\n')


>>> There is multiple sampling frequency for EOGs in your dataset! <<<

	Number of different sampling frequency configuration: 3

Quick overlook of the EOGs associated to sampling frequencies:

256 Hz: ['EOG G' 'EOG D']

400 Hz: ['EOG1' 'EOG2']

128 Hz: ['EOG D' 'EOG G']


In [26]:
# widget to select the configuration of interest
config_sf_slider = mk_config_slider(value = 1, min = 1, max = len(sf_config_dict))

# print the configuration selected
# interact with the slider output through the printing function 
widgets.interact(lambda i: print_config(i, config_dict=sf_config_dict, param="Sampling frequencies"), i=config_sf_slider);

interactive(children=(IntSlider(value=1, continuous_update=False, description='Selected configuration:', layou…

### 5.4 Inspect EOG filtering parameters

In [27]:
if len(df_eog['highpass'].unique())+len(df_eog['lowpass'].unique())+len(df_eog['notch'].unique()) == 3:
    print('\n>>> All EOGs have the same filtering parameters! <<<')
elif len(df_eog['highpass'].unique())+len(df_eog['lowpass'].unique())+len(df_eog['notch'].unique()) > 3:
    print('\n>>> Filtering parameters are not fully consistent across the dataset! <<<')
else:
    print('\n>>> There may have been a problem in reading the filtering parameters. Here is the output: <<<')

# Get the list of participants with different filtering parameters
# 1st replace NaN because groupby does not like NaN
df_filt = df_eog.copy()
df_filt[['lowpass', 'highpass', 'notch']] = df_filt[['lowpass', 'highpass', 'notch']].fillna('missing')

config_filters = (
    df_filt.groupby(['lowpass', 'highpass', 'notch'])['subject']
    .apply(lambda x: sorted(set(x)))
    .reset_index(name = 'subjects')
)

# print filter configuration
print(f'\n\tNumber of different EOG filters configurations: {len(config_filters)}\n')
# print('\nFilters configurations: ')
# r=1
# for row in config_filters.itertuples(index=False):
#     print(f'Configuration #{r} ({len(row.subjects)} participants)')
#     print(f'highpass: {row.highpass}, lowpass: {row.lowpass}, notch: {row.notch}\n')
#     r=r+1


>>> Filtering parameters are not fully consistent across the dataset! <<<

	Number of different EOG filters configurations: 3



In [28]:
# widget to select the configuration of interest
config_filter_slider = mk_config_slider(value = 1, min = 1, max = len(config_filters))

# function to rpint filters configurations
def print_filters(config_slider):
    # get the info from the dataframe
    idx = config_slider - 1
    sID = config_filters.iloc[idx]['subjects']
    hpass = config_filters.iloc[idx]['highpass']
    lpass = config_filters.iloc[idx]['lowpass']
    notch = config_filters.iloc[idx]['notch']
    
    # print info
    print(f'Selected configuration: # {config_slider}')
    print(f'\tFilters configuration: highpass: {hpass}; lowpass: {lpass}; notch: {notch}')
    print(f'\t{len(sID)} participants: {sID}')

widgets.interact(print_filters, config_slider = config_filter_slider);

interactive(children=(IntSlider(value=1, continuous_update=False, description='Selected configuration:', layou…

### 5.5 Inspect EOG units

In [29]:
if len(df_eog['dimension'].unique()) == 1:
    print(f'\n>>> All EOGs have the same unit: {df_eog["dimension"].unique()} <<<\n')
elif len(df_eog['dimension'].unique()) > 1:
    print('\n>>> Multiple units were found! <<<')
    print(f'\n\tNumber of different EOG units configurations: {len(df_eog['dimension'].unique())}\n')
    print('Quick overlook of EOGs associated to units:')
    for u, unit in enumerate(df_eog['dimension'].unique()):
        # select only rows with the current sf
        df_unit = df_eog[df_eog['dimension'] == unit].copy()
        print(f'\n{unit}: {df_unit["channel"].unique()}')
    

# print the different configuration of units 
# if info about sf configuration is needed
unit_per_sub = df_eog.groupby('subject')['dimension'].apply(lambda x: tuple(sorted(set(x))))
ch_per_unit = df_eog.groupby('dimension')['channel'].apply(lambda x: tuple(sorted(set(x))))
# identify the sampling frequency configuration of each participant and store them in a dict to print per sampling configuration config
unit_config_dict = {}
for config in unit_per_sub.unique():
    sub = unit_per_sub[unit_per_sub == config].index.tolist()
    unit_config_dict[config] = sub

# print info per sf configuration
# print('\nUnits configurations:')
# for i, (config, participants) in enumerate(unit_config_dict.items(), 1):
#     print(f'Configuration #{i} ({len(participants)} participants):')
#     print(f'Unit ({len(config)}) : {config}\n')
#     # print(f"Participants : {participants}\n")


>>> Multiple units were found! <<<

	Number of different units configurations: 3

Quick overlook of channels associated to units:

mV: ['EOG G' 'EOG D']

UV: ['EOG1' 'EOG2']

uV: ['EOG D' 'EOG G']


In [30]:
# widget to select the configuration of interest
config_unit_slider = mk_config_slider(value = 1, min = 1, max = len(unit_config_dict))

# print the configuration selected
# interact with the slider output through the printing function 
widgets.interact(lambda i: print_config(i, config_dict=unit_config_dict, param="Units"), i=config_unit_slider);

interactive(children=(IntSlider(value=1, continuous_update=False, description='Selected configuration:', layou…

### 5.6 Inspect EOG signal inversion

In [31]:
# select rows where the physical min is greater than the physical max
df_inv = df_eog[df_eog['physical_min'] > df_eog['physical_max']]

if not df_inv.empty:
    print('\n>>> Inverted polarity detected in EOGs! <<<')
    print(f'{df_inv.shape[0]} EOGs have an inverted polarity (from {df_eog.shape[0]} EOGs in {len(edf_files)} edf files)')
    print(df_inv[['subject', 'channel', 'dimension', 'physical_min', 'physical_max']])
else:
    print('\n>>> No inverted polarity was detected in EOGs <<<')
df_inv.to_csv(f'{summary_path}/EOG_inverted_polarity_edf.tsv', sep = '\t')
print(f'\nSaving informations from inverted polarity EOGs to:\n{summary_path}/EOG_inverted_polarity_edf.tsv \n(will be empty if no inverted polarity)')



>>> No inverted polarity was detected in EOGs <<<

Saving informations from EOG inverted polarity channels to:
C:\Users\yvan.nedelec\OneDrive - ICM\Documents\RE_yvan\projects\ICEBERG\PSG/summary/EOG_inverted_polarity_edf.tsv 
(will be empty if no inverted polarity)


### 5.7 Inspect EOG resolution

In [32]:
# res_theo have been converted to uV, but if dimension was not read or not indicated in the headers, it might not work. I might need to add something more robust
r_thres = widgets.BoundedFloatText(
    value=0.1,
    min=0,
    max=10.0,
    step=0.1,
    style={'description_width': '150px'},  # augmente la largeur de la description
    layout=widgets.Layout(width='230px'),   # ajuste la taille totale du widget si besoin
    description='Resolution threshold (uV):',
    disabled=False
);

# define a function to interact with the widget
def check_bad_res(threshold):
    r_mask = df_eog['res_theoretical'] >= threshold
    bad_res = df_eog[r_mask]
    
    if not bad_res.empty:
        print(f'\n>>> Poor resolution detected in EOGs! (>= {threshold} uV) <<<')
        print(f'{bad_res.shape[0]} EOGs have a very poor resolution (from {df_eog.shape[0]} EOGs in {len(edf_files)} edf files)')
        print(bad_res[['subject', 'channel', 'dimension', 'physical_min', 'physical_max', 'res_theoretical']])
    else:
        print(f'\n>>> No EOG with a poor resolution (>= {threshold} uV) was detected! <<<')
    bad_res.to_csv(f'{summary_path}/EOG_bad_resolution_edf.tsv', sep = '\t')
    print(f'\nSaving informations from bad resolution EOGs to:\n{summary_path}/EOG_bad_resolution_edf.tsv \n(will be empty if no bad resolution)')

widgets.interact(check_bad_res, threshold=r_thres);

interactive(children=(BoundedFloatText(value=0.1, description='Resolution threshold (uV):', layout=Layout(widt…

### 5.8 Inspect EOG dynamic range

In [33]:
dr_thres = widgets.BoundedFloatText(
    value=400,
    min=0,
    max=5000,
    step=0.1,
    style={'description_width': '200px'},  # augmente la largeur de la description
    layout=widgets.Layout(width='270px'),   # ajuste la taille totale du widget si besoin
    description='Dynamic range threshold (uV):',
    disabled=False
);


def check_bad_dr(threshold):
    dr_mask = df_eog['res_theoretical']*pow(2,16) <= threshold
    bad_dr = df_eog[dr_mask]
    
    if not bad_dr.empty:
        print(f'\n>>> Small dynamic range detected in EOGs! (<= {threshold} uV) <<<\n')
        print(f'{bad_dr.shape[0]} EOGs have a small dynamic range (from {df_eog.shape[0]} EOGs in {len(edf_files)} edf files)')
        print(bad_dr[['subject', 'channel', 'dimension', 'physical_min', 'physical_max', 'res_theoretical']])
    else:
        print(f'\n>>> No EOG with a small dynamic range(<= {threshold} uV) was detected! <<<')
    bad_dr.to_csv(f'{summary_path}/EOG_bad_dynamic_range_edf.tsv', sep = '\t')
    print(f'\nSaving informations from bad dynamic range EOGs to:\n{summary_path}/EOG_bad_dynamic_range_edf.tsv \n(will be empty if no bad resolution)')

widgets.interact(check_bad_dr, threshold = dr_thres);

interactive(children=(BoundedFloatText(value=400.0, description='Dynamic range threshold (uV):', layout=Layout…

## 6. Inspect ECG

### 6.1 Select only ECGs

In [35]:
# select only ECGs and return a warning if the number of participant is smaller/higher
mask_ecg = df_full['channel'].str.contains(r'ecg', case = False, na=False) # create a mask that returns true for lines containing either ecg in the channel column
df_ecg = df_full[mask_ecg]

# Check if the number of participants with only ECG is the same as df_full. 
# If not, it might be because the transducer type was no correctly detected. 
# One possibility is to add the type of transducer to the condition line 2 of this cell.
if len(df_full['subject'].unique()) > len(df_ecg['subject'].unique()):
    # identify missing subjects
    missing_sub = set(df_full['subject'].unique()) - set(df_ecg['subject'].unique())
    print('\n!!! There is less participants in the dataset with only ECGs !!!')
    print(f'Missing participants: {missing_sub}')
    print("\nEither these participants don't have ECGs.")
    print("Or the transducer type was not correctly detected.")
    # get df of missing sub to save and inspect
    df_ecgmiss = df_full[df_full['subject'].isin(missing_sub)]
    df_ecgmiss.to_csv(f'{summary_path}/ECG_missing_edf.tsv', sep = '\t')
    print(f'\nSaving informations from missing participants to:\n{summary_path}/ECG_missing_edf.tsv')
    print('Please inspect the file, and specifically the column transducer_type')
elif len(df_full['subject'].unique()) < len(df_ecg['subject'].unique()):
    print('\n!!! There is more participants in the dataset with only ECGs !!!')
    print('This should not be the case.')
    print('Please inspect what is happening in a code editor (spyder..), or ask Yvan.')
    more_sub = set(df_ecg['subject'].unique()) - set(df_full['subject'].unique())
    df_more = df_ecg[df_ecg['subject'].isin(more_sub)]
    df_more.to_csv(f'{summary_path}/ECG_suspect_edf.csv', sep = '\t')
    print(f'\nSaving informations from suspect participants to:\n{summary_path}/ECG_suspect_edf.tsv')

# saving info from ECG
df_ecg.to_csv(f'{summary_path}/ECG_summary_table.tsv', sep = '\t')
print(f'\nSaving informations from ECGs to:\n{summary_path}/ECG_summary_table.tsv')


Saving informations from ECG channels to:
C:\Users\yvan.nedelec\OneDrive - ICM\Documents\RE_yvan\projects\ICEBERG\PSG/summary/ECG_summary_table.tsv


### 6.2 Inspect ECG configurations

In [36]:
# get the ECGs configuration per participant 
ch_per_sub = df_ecg.groupby('subject')['channel'].apply(lambda x: tuple(sorted(set(x))))

# identify the ECG configuration of each participant and store them in a dict to print per ECG config
ch_config_dict = {}
for config in ch_per_sub.unique():
    sub = ch_per_sub[ch_per_sub == config].index.tolist()
    ch_config_dict[config] = sub

if len(ch_config_dict) > 1:
    print('\n>>> There is multiple ECG configurations in your dataset! <<<')    
    print(f'\n\tNumber of different ECG configuration: {len(ch_config_dict)}\n')
else:
    print('\n>>> There is only one ECG configuration in your dataset! <<<')

# # print info per channel configuaration
# for i, (config, participants) in enumerate(ch_config_dict.items(), 1):
#     print(f'Configuration #{i} ({len(participants)} participants):')
#     print(f'Channels ({len(config)}) : {config}\n')


>>> There is multiple ECG configurations in your dataset! <<<

	Number of different ECG configuration: 3



In [37]:
# widget to select the configuration of interest
config_ch_slider = mk_config_slider(value = 1, min = 1, max = len(ch_config_dict))

# print the configuration selected
# interact with the slider output through the printing function 
widgets.interact(lambda i: print_config(i, config_dict=ch_config_dict, param="Channels"), i=config_ch_slider);

interactive(children=(IntSlider(value=1, continuous_update=False, description='Selected configuration:', layou…

### 6.3 Inspect ECG sampling frequencies

In [38]:
# the sampling frequency configuration
sf_per_sub = df_ecg.groupby('subject')['sampling_frequency'].apply(lambda x: tuple(sorted(set(x))))
# identify the sampling frequency configuration of each participant and store them in a dict to print per sampling configuration config
sf_config_dict = {}
for config in sf_per_sub.unique():
    sub = sf_per_sub[sf_per_sub == config].index.tolist()
    sf_config_dict[config] = sub

# print info per sf configuration (maybe print it only for multiple config)
if len(sf_config_dict) > 1:
    print('\n>>> There is multiple sampling frequency for ECGs in your dataset! <<<')    
    print(f'\n\tNumber of different sampling frequency configuration: {len(sf_config_dict)}\n')
    print('Quick overlook of the ECGs associated to sampling frequencies:')
    for s, sf in enumerate(df_ecg['sampling_frequency'].unique()):
        # select only rows with the current sf
        df_sf = df_ecg[df_ecg['sampling_frequency'] == sf].copy()
        print(f'\n{sf} Hz: {df_sf["channel"].unique()}')
else:
    print(f'\n>>> There is only one sampling frequency for ECGs in your dataset: {df_ecg['sampling_frequency'].unique()} <<<')

# print('\nSampling frequency configurations:\n')
# for i, (config, participants) in enumerate(sf_config_dict.items(), 1):
#     print(f'Configuration #{i} ({len(participants)} participants):')
#     print(f'Sampling frequency ({len(config)}) : {config}\n')


>>> There is multiple sampling frequency for ECGs in your dataset! <<<

	Number of different sampling frequency configuration: 3

Quick overlook of the ECGs associated to sampling frequencies:

256 Hz: ['ECG' 'ECG 1' 'ECG 2']

128 Hz: ['ECG 1' 'ECG 2']

400 Hz: ['ECG']


In [39]:
# widget to select the configuration of interest
config_sf_slider = mk_config_slider(value = 1, min = 1, max = len(sf_config_dict))

# print the configuration selected
# interact with the slider output through the printing function 
widgets.interact(lambda i: print_config(i, config_dict=sf_config_dict, param="Sampling frequencies"), i=config_sf_slider);

interactive(children=(IntSlider(value=1, continuous_update=False, description='Selected configuration:', layou…

### 6.4 Inspect ECG filtering parameters

In [40]:
if len(df_ecg['highpass'].unique())+len(df_ecg['lowpass'].unique())+len(df_ecg['notch'].unique()) == 3:
    print('\n>>> All ECGs have the same filtering parameters! <<<')
elif len(df_ecg['highpass'].unique())+len(df_ecg['lowpass'].unique())+len(df_ecg['notch'].unique()) > 3:
    print('\n>>> Filtering parameters are not fully consistent across the dataset! <<<')
else:
    print('\n>>> There may have been a problem in reading the filtering parameters. Here is the output: <<<')

# Get the list of participants with different filtering parameters
# 1st replace NaN because groupby does not like NaN
df_filt = df_ecg.copy()
df_filt[['lowpass', 'highpass', 'notch']] = df_filt[['lowpass', 'highpass', 'notch']].fillna('missing')

config_filters = (
    df_filt.groupby(['lowpass', 'highpass', 'notch'])['subject']
    .apply(lambda x: sorted(set(x)))
    .reset_index(name = 'subjects')
)

# print filter configuration
print(f'\n\tNumber of different ECG filters configurations: {len(config_filters)}\n')
# print('\nFilters configurations: ')
# r=1
# for row in config_filters.itertuples(index=False):
#     print(f'Configuration #{r} ({len(row.subjects)} participants)')
#     print(f'highpass: {row.highpass}, lowpass: {row.lowpass}, notch: {row.notch}\n')
#     r=r+1


>>> Filtering parameters are not fully consistent across the dataset! <<<

	Number of different ECG filters configurations: 3



In [41]:
# widget to select the configuration of interest
config_filter_slider = mk_config_slider(value = 1, min = 1, max = len(config_filters))

# function to rpint filters configurations
def print_filters(config_slider):
    # get the info from the dataframe
    idx = config_slider - 1
    sID = config_filters.iloc[idx]['subjects']
    hpass = config_filters.iloc[idx]['highpass']
    lpass = config_filters.iloc[idx]['lowpass']
    notch = config_filters.iloc[idx]['notch']
    
    # print info
    print(f'Selected configuration: # {config_slider}')
    print(f'\tFilters configuration: highpass: {hpass}; lowpass: {lpass}; notch: {notch}')
    print(f'\t{len(sID)} participants: {sID}')

widgets.interact(print_filters, config_slider = config_filter_slider);

interactive(children=(IntSlider(value=1, continuous_update=False, description='Selected configuration:', layou…

### 6.5 Inspect ECG units

In [42]:
if len(df_ecg['dimension'].unique()) == 1:
    print(f'\n>>> All ECGs have the same unit: {df_ecg["dimension"].unique()} <<<\n')
elif len(df_ecg['dimension'].unique()) > 1:
    print('\n>>> Multiple units were found for ECGs! <<<')
    print(f'\n\tNumber of different units configurations: {len(df_ecg['dimension'].unique())}\n')
    print('Quick overlook of ECGs associated to units:')
    for u, unit in enumerate(df_ecg['dimension'].unique()):
        # select only rows with the current sf
        df_unit = df_ecg[df_ecg['dimension'] == unit].copy()
        print(f'\n{unit}: {df_unit["channel"].unique()}')
    

# print the different configuration of units 
# if info about sf configuration is needed
unit_per_sub = df_ecg.groupby('subject')['dimension'].apply(lambda x: tuple(sorted(set(x))))
ch_per_unit = df_ecg.groupby('dimension')['channel'].apply(lambda x: tuple(sorted(set(x))))
# identify the sampling frequency configuration of each participant and store them in a dict to print per sampling configuration config
unit_config_dict = {}
for config in unit_per_sub.unique():
    sub = unit_per_sub[unit_per_sub == config].index.tolist()
    unit_config_dict[config] = sub

# print info per sf configuration
# print('\nUnits configurations:')
# for i, (config, participants) in enumerate(unit_config_dict.items(), 1):
#     print(f'Configuration #{i} ({len(participants)} participants):')
#     print(f'Unit ({len(config)}) : {config}\n')
#     # print(f"Participants : {participants}\n")


>>> Multiple units were found! <<<

	Number of different units configurations: 2

Quick overlook of channels associated to units:

mV: ['ECG' 'ECG 1' 'ECG 2']

MV: ['ECG']


In [43]:
# widget to select the configuration of interest
config_unit_slider = mk_config_slider(value = 1, min = 1, max = len(unit_config_dict))

# print the configuration selected
# interact with the slider output through the printing function 
widgets.interact(lambda i: print_config(i, config_dict=unit_config_dict, param="Units"), i=config_unit_slider);

interactive(children=(IntSlider(value=1, continuous_update=False, description='Selected configuration:', layou…

### 6.6 Inspect ECG signal inversion

In [45]:
# select rows where the physical min is greater than the physical max
df_inv = df_ecg[df_ecg['physical_min'] > df_ecg['physical_max']]

if not df_inv.empty:
    print('\n>>> Inverted polarity detected in ECGs! <<<')
    print(f'{df_inv.shape[0]} ECGs have an inverted polarity (from {df_ecg.shape[0]} ECGs in {len(edf_files)} edf files)')
    print(df_inv[['subject', 'channel', 'dimension', 'physical_min', 'physical_max']])
else:
    print('\n>>> No inverted polarity was detected in ECGs <<<')
df_inv.to_csv(f'{summary_path}/ECG_inverted_polarity_edf.tsv', sep = '\t')
print(f'\nSaving informations from inverted polarity ECGs to:\n{summary_path}/ECG_inverted_polarity_edf.tsv \n(will be empty if no inverted polarity)')



>>> No inverted polarity was detected in ECGs <<<

Saving informations from ECG inverted polarity channels to:
C:\Users\yvan.nedelec\OneDrive - ICM\Documents\RE_yvan\projects\ICEBERG\PSG/summary/ECG_inverted_polarity_edf.tsv 
(will be empty if no inverted polarity)


### 6.7 Inspect ECG resolution

In [46]:
# res_theo have been converted to uV, but if dimension was not read or not indicated in the headers, it might not work. I might need to add something more robust
r_thres = widgets.BoundedFloatText(
    value=0.1,
    min=0,
    max=10.0,
    step=0.1,
    style={'description_width': '150px'},  # augmente la largeur de la description
    layout=widgets.Layout(width='230px'),   # ajuste la taille totale du widget si besoin
    description='Resolution threshold (uV):',
    disabled=False
);

# define a function to interact with the widget
def check_bad_res(threshold):
    r_mask = df_ecg['res_theoretical'] >= threshold
    bad_res = df_ecg[r_mask]
    
    if not bad_res.empty:
        print(f'\n>>> Poor resolution detected in ECGs! (>= {threshold} uV) <<<')
        print(f'{bad_res.shape[0]} ECGs have a very poor resolution (from {df_ecg.shape[0]} ecgs in {len(edf_files)} edf files)')
        print(bad_res[['subject', 'channel', 'dimension', 'physical_min', 'physical_max', 'res_theoretical']])
    else:
        print(f'\n>>> No ECG with a poor resolution (>= {threshold} uV) was detected! <<<')
    bad_res.to_csv(f'{summary_path}/ECG_bad_resolution_edf.tsv', sep = '\t')
    print(f'\nSaving informations from bad resolution ECGs to:\n{summary_path}/ECG_bad_resolution_edf.tsv \n(will be empty if no bad resolution)')

widgets.interact(check_bad_res, threshold=r_thres);

interactive(children=(BoundedFloatText(value=0.1, description='Resolution threshold (uV):', layout=Layout(widt…

### 6.8 Inspect ECG dynamic range

In [47]:
dr_thres = widgets.BoundedFloatText(
    value=400,
    min=0,
    max=5000,
    step=0.1,
    style={'description_width': '200px'},  # augmente la largeur de la description
    layout=widgets.Layout(width='270px'),   # ajuste la taille totale du widget si besoin
    description='Dynamic range threshold (uV):',
    disabled=False
);


def check_bad_dr(threshold):
    dr_mask = df_ecg['res_theoretical']*pow(2,16) <= threshold
    bad_dr = df_ecg[dr_mask]
    
    if not bad_dr.empty:
        print(f'\n>>> Small dynamic range detected in ECGs! (<= {threshold} uV) <<<\n')
        print(f'{bad_dr.shape[0]} ECGs have a small dynamic range (from {df_ecg.shape[0]} ECGs in {len(edf_files)} edf files)')
        print(bad_dr[['subject', 'channel', 'dimension', 'physical_min', 'physical_max', 'res_theoretical']])
    else:
        print(f'\n>>> No ECG with a small dynamic range(<= {threshold} uV) was detected! <<<')
    bad_dr.to_csv(f'{summary_path}/ECG_bad_dynamic_range_edf.tsv', sep = '\t')
    print(f'\nSaving informations from bad dynamic range ECGs to:\n{summary_path}/ECG_bad_dynamic_range_edf.tsv \n(will be empty if no bad resolution)')

widgets.interact(check_bad_dr, threshold = dr_thres);

interactive(children=(BoundedFloatText(value=400.0, description='Dynamic range threshold (uV):', layout=Layout…